ARTI308 - Machine Learning

# Lab 3: Exploratory Data Analysis (EDA) — House Prices Dataset

**Dataset used:** `house_prices.csv`

This notebook follows the Lab 3 checklist:
- Missing values
- Duplicates
- Shape (rows/columns)
- Data types
- Summary statistics
- Univariate analysis
- Bivariate analysis
- Correlation matrix
- *(Time-based analysis note at the end: this dataset has no date column)*


In [ ]:
# =========================
# 1) Load libraries + dataset
# =========================
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# Read dataset
df = pd.read_csv(r"/mnt/data/house_prices.csv")

# Quick look
display(df.head())
print("Shape:", df.shape)


## Check Missing Values

In [ ]:
# Missing values (count + percentage)
missing_count = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)

missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_%": missing_pct.round(2)
})

display(missing_table[missing_table["missing_count"] > 0])


## Check Duplicate Rows

In [ ]:
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

# If you want to remove duplicates (optional):
# df = df.drop_duplicates().reset_index(drop=True)


## No. of Rows and Columns

In [ ]:
rows, cols = df.shape
print(f"Rows: {rows:,}")
print(f"Columns: {cols}")


## Data Types of Columns

In [ ]:
display(df.dtypes.to_frame("dtype"))


## Descriptive Summary Statistics

In [ ]:
# Numeric columns summary
display(df.describe(numeric_only=True).T)

# Categorical columns quick summary (top values)
cat_cols = df.select_dtypes(include=["object"]).columns
for c in cat_cols[:8]:  # show a few to keep output readable
    print(f"\n--- {c} (top 10) ---")
    display(df[c].value_counts(dropna=False).head(10))


# Feature Cleaning / Preparation (for better plots)

This dataset contains some columns stored as text (examples: `"42 Lac"`, `"1.40 Cr"`, `"500 sqft"`, `"10 out of 11"`).
We will create clean numeric versions:
- `amount_rupees` from `Amount(in rupees)`
- `carpet_sqft` from `Carpet Area`
- `floor_current`, `floor_total` from `Floor`


In [ ]:
def parse_amount_to_rupees(x):
    """Convert strings like '42 Lac', '1.40 Cr' to rupees (float).
    Returns NaN if it cannot parse.
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower().replace(",", "")
    # Extract number part
    m = re.search(r"([0-9]*\.?[0-9]+)", s)
    if not m:
        return np.nan
    val = float(m.group(1))
    # Units
    if "cr" in s or "crore" in s:
        return val * 1e7      # 1 crore = 10,000,000
    if "lac" in s or "lakh" in s:
        return val * 1e5      # 1 lakh = 100,000
    # If already looks like rupees (pure number)
    return val

def parse_sqft(x):
    """Convert '500 sqft' -> 500. Returns NaN if cannot parse."""
    if pd.isna(x):
        return np.nan
    s = str(x).lower().replace(",", "").strip()
    m = re.search(r"([0-9]*\.?[0-9]+)", s)
    return float(m.group(1)) if m else np.nan

def parse_floor(x):
    """Convert '10 out of 11' -> (10, 11). Returns (NaN, NaN) if cannot parse."""
    if pd.isna(x):
        return (np.nan, np.nan)
    s = str(x).lower().strip()
    m = re.search(r"(\d+)\s*out\s*of\s*(\d+)", s)
    if not m:
        return (np.nan, np.nan)
    return (float(m.group(1)), float(m.group(2)))

# Create cleaned columns
df["amount_rupees"] = df["Amount(in rupees)"].apply(parse_amount_to_rupees)
df["carpet_sqft"] = df["Carpet Area"].apply(parse_sqft)

floor_parsed = df["Floor"].apply(parse_floor)
df["floor_current"] = floor_parsed.apply(lambda t: t[0])
df["floor_total"] = floor_parsed.apply(lambda t: t[1])

# Also keep numeric 'Price (in rupees)' (already numeric but has missing values)
# We'll focus on 'amount_rupees' as the total listing amount, if present.
display(df[["Amount(in rupees)", "amount_rupees", "Carpet Area", "carpet_sqft", "Floor", "floor_current", "floor_total"]].head(10))
print("Cleaned columns created.")


## Univariate Analysis

### Distribution of Total Amount (amount_rupees)

In [ ]:
# Use a trimmed view to make plots readable (remove extreme outliers for visualization only)
amt = df["amount_rupees"].dropna()
if len(amt) > 0:
    p1, p99 = np.percentile(amt, [1, 99])
    amt_trim = amt[(amt >= p1) & (amt <= p99)]

    plt.figure(figsize=(8,4))
    plt.hist(amt_trim, bins=50)
    plt.title("Distribution of Amount (trimmed 1%–99%)")
    plt.xlabel("Amount (rupees)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

    print("Amount summary (rupees):")
    display(amt.describe(percentiles=[.25,.5,.75,.9,.95,.99]).to_frame().T)
else:
    print("No parsable values in amount_rupees.")


### Distribution of Carpet Area (sqft)

In [ ]:
carpet = df["carpet_sqft"].dropna()
if len(carpet) > 0:
    p1, p99 = np.percentile(carpet, [1, 99])
    carpet_trim = carpet[(carpet >= p1) & (carpet <= p99)]

    plt.figure(figsize=(8,4))
    plt.hist(carpet_trim, bins=50)
    plt.title("Distribution of Carpet Area (trimmed 1%–99%)")
    plt.xlabel("Carpet area (sqft)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()
else:
    print("No parsable values in carpet_sqft.")


### Top 10 Locations by number of listings

In [ ]:
top_locations = df["location"].value_counts().head(10)
display(top_locations.to_frame("count"))

plt.figure(figsize=(10,4))
plt.bar(top_locations.index.astype(str), top_locations.values)
plt.title("Top 10 Locations by Listings Count")
plt.xlabel("Location")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


## Bivariate Analysis

### Average Amount by Location (Top 10 by listings)

In [ ]:
loc_top10 = df["location"].value_counts().head(10).index
tmp = df[df["location"].isin(loc_top10)].copy()

avg_by_loc = tmp.groupby("location")["amount_rupees"].mean().sort_values(ascending=False)
display(avg_by_loc.to_frame("avg_amount_rupees"))

plt.figure(figsize=(10,4))
plt.bar(avg_by_loc.index.astype(str), avg_by_loc.values)
plt.title("Average Listing Amount by Location (Top 10 locations)")
plt.xlabel("Location")
plt.ylabel("Average amount (rupees)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


### Average Amount by Status

In [ ]:
avg_by_status = df.groupby("Status")["amount_rupees"].mean().sort_values(ascending=False)
display(avg_by_status.to_frame("avg_amount_rupees"))

plt.figure(figsize=(10,4))
plt.bar(avg_by_status.index.astype(str), avg_by_status.values)
plt.title("Average Listing Amount by Status")
plt.xlabel("Status")
plt.ylabel("Average amount (rupees)")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.show()


### Carpet Area vs Amount (relationship)

In [ ]:
# Scatter with trimming for readability
plot_df = df[["carpet_sqft", "amount_rupees"]].dropna()
if len(plot_df) > 0:
    # Trim for plotting
    a = plot_df["amount_rupees"].values
    c = plot_df["carpet_sqft"].values
    a1, a99 = np.percentile(a, [1, 99])
    c1, c99 = np.percentile(c, [1, 99])
    plot_df = plot_df[(plot_df["amount_rupees"].between(a1, a99)) & (plot_df["carpet_sqft"].between(c1, c99))]

    plt.figure(figsize=(7,5))
    plt.scatter(plot_df["carpet_sqft"], plot_df["amount_rupees"], s=8, alpha=0.4)
    plt.title("Carpet Area vs Amount (trimmed 1%–99%)")
    plt.xlabel("Carpet area (sqft)")
    plt.ylabel("Amount (rupees)")
    plt.tight_layout()
    plt.show()
else:
    print("Not enough numeric data for scatter plot.")


## Correlation Matrix

In [ ]:
# Select numeric columns for correlation
num_cols = ["amount_rupees", "Price (in rupees)", "carpet_sqft", "Bathroom", "Balcony", "floor_current", "floor_total"]
corr_df = df[num_cols].copy()

corr = corr_df.corr(numeric_only=True)

plt.figure(figsize=(8,6))
plt.imshow(corr, aspect="auto")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.index)), corr.index)
plt.title("Correlation Matrix (numeric features)")
plt.colorbar()
plt.tight_layout()
plt.show()

display(corr)


## Time-Based Analysis (Monthly Trend)

This dataset does **not** include a date/time column (e.g., listing date or transaction date), so a **true monthly trend** cannot be computed.

✅ If you have a dataset that contains a date column, we can add:
- convert to datetime
- extract month/year
- group by month and plot trend

For now, this section is intentionally noted as not applicable for this dataset.


# Conclusion

We completed the EDA checklist:
- Identified missing values and duplicates
- Checked shape and data types
- Produced summary statistics for numeric + categorical data
- Built cleaned numeric features for better analysis
- Performed univariate and bivariate analysis with visualizations
- Computed a correlation matrix for key numeric features
